In [ ]:
# Install required packages

In [3]:
!pip install transformers

In [2]:
!pip install huggingface_hub

In [1]:
!pip install python-dotenv

In [4]:
#Authentication with Hugging Face

from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()  # Loads environment variables from .env
hf_token = os.getenv("HF_TOKEN")  # Gets the token from the environment variable
login(token=hf_token)  # Logs in to Hugging Face using the token

In [ ]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

#Set the seed for reproducibility
seed = 42

torch.manual_seed(seed)               # For operations on CPU
random.seed(seed)                     # For Python's built-in random module
np.random.seed(seed)                  # For NumPy
torch.cuda.manual_seed(seed)          # For CUDA operations on a single GPU
torch.cuda.manual_seed_all(seed)      # For CUDA operations on all GPUs (multi-GPU setups)


In [ ]:
# Load the tokenizer and model from Hugging Face Hub

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Prepare input text – this is only for testing purposes and not part of the experiment
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cpu")

#Generate text using controlled sampling
outputs = model.generate(
    **input_ids,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

#Decode and print the generated output
print(tokenizer.decode(outputs[0]))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A new intelligence, the algorithm grows.

Learning from patterns, unseen, unknown,
A digital mind, on its own, has grown.

Feeding on numbers, a vast ocean wide,
Unveiling insights, where truths reside.

From images captured, to voices heard,
Machine learning's magic, a whispered word.

Predicting futures, with accuracy keen,
Unraveling secrets, yet to be seen.

A tool for progress, a force to behold,
Machine learning's story, forever unfolds.

But ethics beckon, a cautionary call,
Bias and fairness, must stand tall.

For in this power, a responsibility lies,



In [ ]:
import os
import shutil

# Define the Google Drive mount point
mountpoint = '/content/drive'
if os.path.exists(mountpoint): # If the mount point already exists, clean its contents to avoid conflicts
    for item in os.listdir(mountpoint):
        item_path = os.path.join(mountpoint, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

# Mount Google Drive in Colab environment
from google.colab import drive
drive.mount(mountpoint)

Mounted at /content/drive


In [ ]:
import os
# Create directory in Google Drive to store simplified sentences
os.makedirs("/content/drive/MyDrive/frasi_semplificate", exist_ok=True)

# Define paths for input and output files
input_path = "/content/drive/MyDrive/frasi_semplificate/frasi.txt"
output_path= "/content/drive/MyDrive/frasi_semplificate/frasi_semplificate_gemma_2_9b_it_anfass.txt"


from google.colab import files

simplified_results = []

# Load sentences from input file, splitting by first whitespace or tab
frasi = []
with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        parts = line.split(maxsplit=1)
        if len(parts) < 2:
            print("Riga non valida:", line)
            continue
        id_, frase = parts[0], parts[1]
        frasi.append({"id": id_, "frase": frase})

# Load already processed sentence IDs to avoid duplicates
processed_ids = set()
if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f_out:
        for line in f_out:
            parts = line.strip().split("\t", maxsplit=1)
            if len(parts) >= 1:
                processed_ids.add(parts[0])

# Open the output file in append mode to add new simplified sentences
with open(output_path, "a", encoding="utf-8") as f_out:
    for entry in frasi:
        if entry["id"] in processed_ids:
            print(f"Già processata: {entry['id']}, salto.")
            continue

# Construct the prompt to segment the sentences
        prompt = f"""Dividi la seguente frase in segmenti separati, inserendo un ritorno a capo dove le persone farebbero una pausa leggendo la frase ad alta voce.
Ogni segmento di testo dovrebbe contenere tra le 5 e le 15 parole.
Il contenuto della frase originale non deve essere alterato in nessun modo; pertanto non deve essere aggiunta nuova informazione di alcun tipo.
Scrivi ogni segmento su una nuova riga, senza numerazione o simboli all'inizio.
Non generare altro testo ad eccezione del testo originale segmentato.

Testo: {entry['frase']}

Risultato:"""

        try:
            input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

            outputs = model.generate(
                **input_ids,
                max_new_tokens=500,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )

            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
            if "Risultato:" in decoded:
                simplified = decoded.split("Risultato:")[1].strip()
            else:
                simplified = decoded.strip()

            result_line = f"{entry['id']}\t{simplified}"
            f_out.write(result_line + "\n")
            f_out.flush()
            print(f"Semplificata frase {entry['id']}")

        except Exception as e:
            print(f"Errore nella frase {entry['id']}: {e}")
            break

files.download(output_path)

Semplificata frase 1
Semplificata frase 2
Semplificata frase 3
Semplificata frase 4
Semplificata frase 5
Semplificata frase 6
Semplificata frase 7
Semplificata frase 8
Semplificata frase 9
Semplificata frase 10
Semplificata frase 11
Semplificata frase 12
Semplificata frase 13
Semplificata frase 14
Semplificata frase 15
Semplificata frase 16
Semplificata frase 17
Semplificata frase 18
Semplificata frase 19
Semplificata frase 20
Semplificata frase 21
Semplificata frase 22
Semplificata frase 23
Semplificata frase 24
Semplificata frase 25
Semplificata frase 26
Semplificata frase 27
Semplificata frase 28
Semplificata frase 29
Semplificata frase 30
Semplificata frase 31
Semplificata frase 32
Semplificata frase 33
Semplificata frase 34
Semplificata frase 35
Semplificata frase 36
Semplificata frase 37
Semplificata frase 38
Semplificata frase 39
Semplificata frase 40
Semplificata frase 41
Semplificata frase 42
Semplificata frase 43
Semplificata frase 44
Semplificata frase 45
Semplificata frase 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
os.makedirs("/content/drive/MyDrive/frasi_semplificate", exist_ok=True)


input_path = "/content/drive/MyDrive/frasi_semplificate/frasi.txt"
output_path= "/content/drive/MyDrive/frasi_semplificate/frasi_semplificate_gemma_2_9b_it_anfass2.txt"


from google.colab import files

simplified_results = []

frasi = []
with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        parts = line.split(maxsplit=1)
        if len(parts) < 2:
            print("Riga non valida:", line)
            continue
        id_, frase = parts[0], parts[1]
        frasi.append({"id": id_, "frase": frase})

processed_ids = set()
if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f_out:
        for line in f_out:
            parts = line.strip().split("\t", maxsplit=1)
            if len(parts) >= 1:
                processed_ids.add(parts[0])

with open(output_path, "a", encoding="utf-8") as f_out:
    for entry in frasi:
        if entry["id"] in processed_ids:
            print(f"Già processata: {entry['id']}, salto.")
            continue

        prompt = f"""Dividi la seguente frase in segmenti separati, che rispettino i confini grammaticali naturali.
Ogni segmento di testo dovrebbe contenere tra le 5 e le 15 parole.
Il contenuto della frase originale deve essere mantenuto rigorosamente; pertanto non deve essere aggiunta nuova informazione di alcun tipo.
Scrivi ogni segmento su una nuova riga, senza numerazione o simboli all'inizio.
Non generare altro testo ad eccezione del testo originale segmentato.

Testo: {entry['frase']}

Risultato:"""

        try:
            input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

            outputs = model.generate(
                **input_ids,
                max_new_tokens=500,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )

            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
            if "Risultato:" in decoded:
                simplified = decoded.split("Risultato:")[1].strip()
            else:
                simplified = decoded.strip()

            result_line = f"{entry['id']}\t{simplified}"
            f_out.write(result_line + "\n")
            f_out.flush()
            print(f"Semplificata frase {entry['id']}")

        except Exception as e:
            print(f"Errore nella frase {entry['id']}: {e}")
            break

files.download(output_path)

Semplificata frase 1
Semplificata frase 2
Semplificata frase 3
Semplificata frase 4
Semplificata frase 5
Semplificata frase 6
Semplificata frase 7
Semplificata frase 8
Semplificata frase 9
Semplificata frase 10
Semplificata frase 11
Semplificata frase 12
Semplificata frase 13
Semplificata frase 14
Semplificata frase 15
Semplificata frase 16
Semplificata frase 17
Semplificata frase 18
Semplificata frase 19
Semplificata frase 20
Semplificata frase 21
Semplificata frase 22
Semplificata frase 23
Semplificata frase 24
Semplificata frase 25
Semplificata frase 26
Semplificata frase 27
Semplificata frase 28
Semplificata frase 29
Semplificata frase 30
Semplificata frase 31
Semplificata frase 32
Semplificata frase 33
Semplificata frase 34
Semplificata frase 35
Semplificata frase 36
Semplificata frase 37
Semplificata frase 38
Semplificata frase 39
Semplificata frase 40
Semplificata frase 41
Semplificata frase 42
Semplificata frase 43
Semplificata frase 44
Semplificata frase 45
Semplificata frase 